### Hybrid Search Langchain

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file
load_dotenv()

# Retrieve the API key from the environment
api_key = os.getenv("PINECONE_KEY")

In [2]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [3]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [4]:
index=pc.Index(index_name)
index

In [5]:
## vector embedding and sparse matrix
import os
from huggingface_hub import login
from langchain_huggingface import HuggingFaceEmbeddings
# Get the token from .env
token = os.getenv("HUGGINGFACE_TOKEN")

if token:
    # Login to Hugging Face
    login(token=token)
    print("Successfully logged in to Hugging Face.")
else:
    print("Token not found. Please check your .env file.")


embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

Successfully logged in to Hugging Face.


HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [6]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

100% [........................................................................] 65406227 / 65406227

In [12]:
sentences=[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
]

## tfidf created on these sentence as vocubalary
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("./Week_3/Day_1/bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("./Week_3/Day_1/bm25_values.json")


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [14]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x2b6efeb60>, index=<pinecone.data.index.Index object at 0x10322beb0>)

In [15]:
retriever.add_texts(
    sentences
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
print(retriever.invoke("When did I visit New Orleans France"))

[Document(metadata={'score': 0.506250322}, page_content='In 2021, I visited New Orleans'), Document(metadata={'score': 0.262276411}, page_content='In 2023, I visited Paris'), Document(metadata={'score': 0.223837838}, page_content='In 2022, I visited New York')]
